In [7]:
# Save Model Using Pickle
import pandas as pd
from sklearn import model_selection
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
#bring in the CSVs
oct19 = pd.read_csv("../data/2019-Oct.csv")
nov19 = pd.read_csv("../data/2019-Nov.csv")
dec19 = pd.read_csv("../data/2019-Dec.csv")
jan20 = pd.read_csv("../data/2020-Jan.csv")
feb20 = pd.read_csv("../data/2020-Feb.csv")
ecom_df = pd.concat([oct19, nov19, dec19, jan20, feb20], axis=0)

#preprocessing
ecom_df = ecom_df[ecom_df.event_type != 'view']
ecom_df = ecom_df[ecom_df.event_type != 'cart']
ecom_df.brand.fillna(value="unknown", axis=0, inplace=True)
ecom_df.dropna(inplace=True)
ecom_df.drop("category_id", axis=1, inplace = True)
ecom_df = ecom_df[ecom_df['price']>= 0]
ecom_df.event_type = ecom_df["event_type"].replace({'purchase': 1, 'remove_from_cart': 0})

#FE
ecom_df['event_time'] = pd.to_datetime(ecom_df['event_time'])
ecom_df['month'] = pd.DatetimeIndex(ecom_df['event_time']).month
ecom_df['2020'] = pd.DatetimeIndex(ecom_df['event_time']).year
ecom_df['hour'] = pd.DatetimeIndex(ecom_df['event_time']).hour
ecom_df['2020'] = ecom_df['2020'].replace({2019: 0, 2020: 1})
ecom_df['day'] = ecom_df['event_time'].dt.dayofweek
ecom_df['day'] = ecom_df['day'].replace({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})
when_df = pd.DataFrame({'hour':range(1, 25)})
bins = [0,6,12,18,24]
labels = ['Night', 'Morning','Afternoon','Evening']
when_df['when'] = pd.cut(when_df['hour'], bins=bins, labels=labels, include_lowest=True)
#may have to fix bellow
ecom_df = ecom_df.join(when_df['when'], on="hour")
ecom_df['month'] = ecom_df['month'].replace({1: 'January', 2: 'Febuary', 10: 'October', 11: 'November', 12: 'December'})

#Labeling
encoder = LabelBinarizer()
lb1_labels = pd.DataFrame(encoder.fit_transform(ecom_df[['brand']]), columns=encoder.classes_, index=ecom_df.index)
encoder = LabelBinarizer()
lb2_labels = pd.DataFrame(encoder.fit_transform(ecom_df[["product_id"]]), columns=encoder.classes_, index=ecom_df.index)
#ohe
ohe = OneHotEncoder(sparse=False, drop='first')
ohe_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['category_code', 'day', 'month', 'when']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[['category_code', 'day', 'month', 'when']].columns))
#join
ecom_df = ecom_df.join(lb1_labels)
ecom_df = ecom_df.join(lb2_labels)
ecom_df = ecom_df.join(ohe_labels)


X = ecom_df.drop(["month", "hour", "day", "when", "user_id", "product_id", "category_code", "event_time", "event_type", "brand", "user_session"], axis=1)
y = ecom_df.event_type
test_size = 0.25
seed = 42
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=test_size, random_state=seed)
# Fit the model on training set
model = RandomForestClassifier()
model.fit(X_train, y_train)
# save the model to disk
filename = 'best_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.6733185513673319
